# Problem Statement

Given the original and degraded versions of a few images. The task is to write a GAN which can fix the degraded images.

Completed the function `fix` at the end of the "Evaluation" block so that it can take a degraded image, and return a fixed image (that looks as much like the original non-degraded version as possible). Read the doc-string of the fix function to see the format.



#Setup

## Intended Structure after Setup

Run the blocks in this section to get the following directory structure:
```
/content
│
└───rephrase-pubfig831
    │
    └───correct
    │   │
    │   └───train
    │   │   │
    │   │   └───Adam Sandler
    │   │   │   │   train__000001-000000.jpg
    │   │   │   │   train__000001-000001.jpg
    │   │   │   │   train__000001-000002.jpg
    │   │   │   │   ...
    │   │   │
    │   │   └───Alec Baldwin
    │   │   │   │   train__000002-000000.jpg
    │   │   │   │   train__000002-000001.jpg
    │   │   │   │   ...
    │   │   │
    │   │   └───Angelina Jolie
    │   │   │   │   train__000003-000000.jpg
    │   │   │   │   train__000003-000001.jpg
    │   │   │   │   ...
    │   │   │
    │   │   │ ...
    │   │
    │   └───test
    │       │
    │       └───Adam Sandler
    │       │   │   test__000001-000000.jpg
    │       │   │   test__000001-000001.jpg
    │       │   │   ...
    │       │
    │       └───Alec Baldwin
    │       │   │   test__000002-000000.jpg
    │       │   │   ...
    │       │
    │       └───Angelina Jolie
    │       │   │   test__000003-000000.jpg
    │       │   │   ...
    │       │
    │       │ ...
    │
    │
    └───degraded
        │   <Same directory structure as 'correct'>
```

Every image in the degraded directory is a degraded version of the image with the same name in the correct directory. e.g. `/content/rephrase-pubfig831/degraded/Adam Sandler/train__000001-000002.jpg` is the degraded version of `/content/rephrase-pubfig831/correct/Adam Sandler/train__000001-000002.jpg`

## Installation (pip etc)
Add any other installation commands you want to in this block.

In [0]:
!pip install GPUtil
!pip install tqdm
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi

  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=acefa72e253209be8524ad04f797eb9342c56ff218e53f92c71a38aa9234dd81
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


## Downloading and Generating Dataset
Run this block only once. Do not modify it. Also, don't call the degrade function in your code anywhere. You should treat the degradation process as a black box.

In [0]:
import os
from glob import glob

import cv2
import numpy as np
from tqdm import tqdm

def degrade(path: str) -> None:
    """Load image at `input_path`, distort and save as `output_path`"""
    SHIFT = 2
    image = cv2.imread(path)
    to_swap = np.random.choice([False, True], image.shape[:2], p=[.8, .2])
    swap_indices = np.where(to_swap[:-SHIFT] & ~to_swap[SHIFT:])
    swap_vals = image[swap_indices[0] + SHIFT, swap_indices[1]]
    image[swap_indices[0] + SHIFT, swap_indices[1]] = image[swap_indices]
    image[swap_indices] = swap_vals
    cv2.imwrite(path, image)

!wget http://briancbecker.com/files/downloads/pubfig83lfw/pubfig83lfw_raw_in_dirs.zip
!unzip -q pubfig83lfw_raw_in_dirs.zip
!rm pubfig83lfw_raw_in_dirs.zip
!mkdir rephrase-pubfig831
!mv pubfig83lfw_raw_in_dirs rephrase-pubfig831/correct
!rm -r rephrase-pubfig831/correct/distract
!cp -r rephrase-pubfig831/correct rephrase-pubfig831/degraded

for image_path in tqdm(glob('rephrase-pubfig831/degraded/*/*/*.jpg')):
  degrade(image_path)

--2019-11-25 14:38:01--  http://briancbecker.com/files/downloads/pubfig83lfw/pubfig83lfw_raw_in_dirs.zip
Resolving briancbecker.com (briancbecker.com)... 162.241.216.158
Connecting to briancbecker.com (briancbecker.com)|162.241.216.158|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400247922 (382M) [application/zip]
Saving to: ‘pubfig83lfw_raw_in_dirs.zip’

pubfig83lfw_raw_in_ 100%[===================>] 381.71M  97.4MB/s    in 4.1s    

2019-11-25 14:38:05 (92.5 MB/s) - ‘pubfig83lfw_raw_in_dirs.zip’ saved [400247922/400247922]



 39%|███▉      | 5130/13002 [00:24<00:36, 218.41it/s]

# **Checking Free Memory**
This block is just so that you can have an idea of the resources you have at hand on the Google Collab system.

In [0]:
import psutil
import humanize
import os
import GPUtil as GPU
gpu = GPU.getGPUs()[0]
process = psutil.Process(os.getpid())
print(f"Gen RAM: Free {humanize.naturalsize(psutil.virtual_memory().available)} | Proc size {humanize.naturalsize(process.memory_info().rss)}")
print(f"GPU RAM: Free {gpu.memoryFree:.0f}MB | Used {gpu.memoryUsed:.0f}MB | Util {gpu.memoryUtil*100:.0f}% | Total {gpu.memoryTotal:.0f}MB")

Gen RAM: Free 12.8 GB | Proc size 177.0 MB
GPU RAM: Free 16280MB | Used 0MB | Util 0% | Total 16280MB


# **Main Code**

## Data Loading

In [0]:
from PIL import Image
import torch
import torch.utils.data as data
import os
import random
from itertools import chain
from torchvision import datasets, models
import torchvision.transforms as transforms
import torch.nn.init as init
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

def prepend(list, str):       
    # Using format() 
    str += '{0}'
    list = [str.format(i) for i in list] 
    return(list)
    
class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir, subpath = ['correct/', 'degraded/'], subfolder='train', direction='AtoB'):
        super(DatasetFromFolder, self).__init__()
        
        self.input_path_a = os.path.join(image_dir, subpath[0], subfolder)
        self.classes_a = [os.path.join(self.input_path_a,x) for x in sorted(os.listdir(self.input_path_a))]
        self.image_filenames_a = []
        for class_ in self.classes_a:
            self.image_filenames_a.append(prepend(sorted(os.listdir(class_)), class_ + '/'))
        self.image_filenames_a = list(chain.from_iterable(self.image_filenames_a))
        
        self.image_filenames_b = []
        for files in self.image_filenames_a:
            files = files.split('/')
            files[1] = subpath[1][:-1]
            files = '/'.join(files)
            self.image_filenames_b.append(files)
        assert len(self.image_filenames_b) == len(self.image_filenames_a), "number of files are differemt"
        self.direction = direction
        
    def __getitem__(self, index):
        # Load Image        
        a = Image.open(self.image_filenames_a[index])
        b = Image.open(self.image_filenames_b[index])   
        
        a = a.resize((286, 286), Image.BICUBIC)
        b = b.resize((286, 286), Image.BICUBIC)
        a = transforms.ToTensor()(a)
        b = transforms.ToTensor()(b)
        w_offset = random.randint(0, max(0, 286 - 256 - 1))
        h_offset = random.randint(0, max(0, 286 - 256 - 1))
    
        a = a[:, h_offset:h_offset + 256, w_offset:w_offset + 256]
        b = b[:, h_offset:h_offset + 256, w_offset:w_offset + 256]
    
        a = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(a)
        b = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(b)

        if random.random() < 0.5:
            idx = [i for i in range(a.size(2) - 1, -1, -1)]
            idx = torch.LongTensor(idx)
            a = a.index_select(2, idx)
            b = b.index_select(2, idx)

        if self.direction == "AtoB":
            return a, b
        else:
            return b, a

    def __len__(self):
        return len(self.image_filenames_a)

## Structure

### **Constants and Hyperparemeters**

In [0]:
dataset = 'rephrase-pubfig831'
subpath = ['correct/', 'degraded/']
subfolder='train'
direction = "BtoA"
filters_gen = 16
filters_dis = 16
input_size = 256
resize = 286
crop_size = 256
fliplr = True
num_epochs = 40
DISCRIMINATOR_FINAL_FEATURE_MAP_SIZE = 10
RESIUDAL_BLOCKS = 16
UPSAMPLING_BLOCKS = 2
lr = 0.001
batch_size = 1
lr_b1 = 0.9
lr_b2 = 0.999



# Directory for saving the model
model_dir = dataset + '/' + '_model/'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

### Generator Model

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision.models import vgg19
import math

class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        vgg19_model = vgg19(pretrained=True)
        self.vgg19_54 = nn.Sequential(*list(vgg19_model.features.children())[:35])

    def forward(self, img):
        return self.vgg19_54(img)

class DenseResidualBlock(nn.Module):
    def __init__(self, filters, res_scale=0.2):
        super(DenseResidualBlock, self).__init__()
        self.res_scale = res_scale

        def block(in_features, non_linearity=True):
            layers = [nn.Conv2d(in_features, filters, 3, 1, 1, bias=True)]
            if non_linearity:
                layers += [nn.LeakyReLU()]
            return nn.Sequential(*layers)

        self.b1 = block(in_features=1 * filters)
        self.b2 = block(in_features=2 * filters)
        self.b3 = block(in_features=3 * filters)
        self.b4 = block(in_features=4 * filters)
        self.b5 = block(in_features=5 * filters, non_linearity=False)
        self.blocks = [self.b1, self.b2, self.b3, self.b4, self.b5]

    def forward(self, x):
        inputs = x
        for block in self.blocks:
            out = block(inputs)
            inputs = torch.cat([inputs, out], 1)
        return out.mul(self.res_scale) + x

class ResidualInResidualDenseBlock(nn.Module):
    def __init__(self, filters, res_scale=0.2):
        super(ResidualInResidualDenseBlock, self).__init__()
        self.res_scale = res_scale
        self.dense_blocks = nn.Sequential(
            DenseResidualBlock(filters), DenseResidualBlock(filters), DenseResidualBlock(filters)
        )

    def forward(self, x):
        return self.dense_blocks(x).mul(self.res_scale) + x


class GeneratorRRDB(nn.Module):
    def __init__(self, channels, filters=16, num_res_blocks=4, num_upsample=2):
        super(GeneratorRRDB, self).__init__()

        # First layer
        self.conv1 = nn.Conv2d(channels, filters, kernel_size=3, stride=1, padding=1)
        # Residual blocks
        self.res_blocks = nn.Sequential(*[ResidualInResidualDenseBlock(filters) for _ in range(num_res_blocks)])
        # Second conv layer post residual blocks
        self.conv2 = nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=1)
        # Upsampling layers
        upsample_layers = []
        for _ in range(num_upsample):
            upsample_layers += [
                nn.Conv2d(filters, filters * 4, kernel_size=3, stride=2, padding=1),
                nn.LeakyReLU(),
                nn.PixelShuffle(upscale_factor=2),
            ]
        self.upsampling = nn.Sequential(*upsample_layers)
        # Final output block
        self.conv3 = nn.Sequential(
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(filters, channels, kernel_size=3, stride=1, padding=1),
        )

    def forward(self, x):
        out1 = self.conv1(x)
        out = self.res_blocks(out1)
        out2 = self.conv2(out)
        out = torch.add(out1, out2)
        out = self.upsampling(out)
        out = self.conv3(out)
        return out


### Discriminator Model

In [0]:

class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        self.input_shape = input_shape
        in_channels, in_height, in_width = self.input_shape
        patch_h, patch_w = int(in_height / 2 ** 4), int(in_width / 2 ** 4)
        self.output_shape = (1, patch_h, patch_w)

        def discriminator_block(in_filters, out_filters, first_block=False):
            layers = []
            layers.append(nn.Conv2d(in_filters, out_filters, kernel_size=3, stride=1, padding=1))
            if not first_block:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            layers.append(nn.Conv2d(out_filters, out_filters, kernel_size=3, stride=2, padding=1))
            layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        layers = []
        in_filters = in_channels
        for i, out_filters in enumerate([64, 128, 256, 512]):
            layers.extend(discriminator_block(in_filters, out_filters, first_block=(i == 0)))
            in_filters = out_filters

        layers.append(nn.Conv2d(out_filters, 1, kernel_size=3, stride=1, padding=1))

        self.model = nn.Sequential(*layers)

    def forward(self, img):
        return self.model(img)
"""


def sigmoid_mul(x):
    return x * F.sigmoid(x)

class FeatureExtractor(nn.Module):
    def __init__(self, cnn, feature_layer=11):
        super(FeatureExtractor, self).__init__()
        self.features = nn.Sequential(*list(cnn.features.children())[:(feature_layer+1)])

    def forward(self, x):
        return self.features(x)


class residualBlock(nn.Module):
    def __init__(self, in_channels=64, k=3, n=64, s=1):
        super(residualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, n, k, stride=s, padding=1)
        self.bn1 = nn.BatchNorm2d(n)
        self.conv2 = nn.Conv2d(n, n, k, stride=s, padding=1)
        self.bn2 = nn.BatchNorm2d(n)

    def forward(self, x):
        y = sigmoid_mul(self.bn1(self.conv1(x)))
        return self.bn2(self.conv2(y)) + x

class upsampleBlock(nn.Module):
    # Implements resize-convolution
    def __init__(self, in_channels, out_channels):
        super(upsampleBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 3, stride=1, padding=1)
        self.shuffler = nn.PixelShuffle(2)

    def forward(self, x):
        return sigmoid_mul(self.shuffler(self.conv(x)))

class Generator(nn.Module):
    def __init__(self, n_residual_blocks, upsample_factor):
        super(Generator, self).__init__()
        self.n_residual_blocks = n_residual_blocks
        self.upsample_factor = upsample_factor

        self.conv1 = nn.Conv2d(3, 64, 9, stride=1, padding=4)

        for i in range(self.n_residual_blocks):
            self.add_module('residual_block' + str(i+1), residualBlock())

        self.conv2 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        for i in range(int(self.upsample_factor/2)):
            self.add_module('upsample' + str(i+1), upsampleBlock(64, 256))

        self.conv3 = nn.Conv2d(64, 3, 9, stride=1, padding=4)

    def forward(self, x):
        x = sigmoid_mul(self.conv1(x))

        y = x.clone()
        for i in range(self.n_residual_blocks):
            y = self.__getattr__('residual_block' + str(i+1))(y)

        x = self.bn2(self.conv2(y)) + x

        for i in range(int(self.upsample_factor/2)):
            x = self.__getattr__('upsample' + str(i+1))(x)

        return self.conv3(x)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)

        self.conv2 = nn.Conv2d(64, 64, 3, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, 3, stride=2, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256, 512, 3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(512)
        self.conv8 = nn.Conv2d(512, 512, 3, stride=2, padding=1)
        self.bn8 = nn.BatchNorm2d(512)

        # Replaced original paper FC layers with FCN
        self.conv9 = nn.Conv2d(512, 1, 1, stride=1, padding=1)

    def forward(self, x):
        x = sigmoid_mul(self.conv1(x))

        x = sigmoid_mul(self.bn2(self.conv2(x)))
        x = sigmoid_mul(self.bn3(self.conv3(x)))
        x = sigmoid_mul(self.bn4(self.conv4(x)))
        x = sigmoid_mul(self.bn5(self.conv5(x)))
        x = sigmoid_mul(self.bn6(self.conv6(x)))
        x = sigmoid_mul(self.bn7(self.conv7(x)))
        x = sigmoid_mul(self.bn8(self.conv8(x)))

        x = self.conv9(x)
        return F.sigmoid(F.avg_pool2d(x, x.size()[2:])).view(x.size()[0], -1)
"""

"\n\n\ndef sigmoid_mul(x):\n    return x * F.sigmoid(x)\n\nclass FeatureExtractor(nn.Module):\n    def __init__(self, cnn, feature_layer=11):\n        super(FeatureExtractor, self).__init__()\n        self.features = nn.Sequential(*list(cnn.features.children())[:(feature_layer+1)])\n\n    def forward(self, x):\n        return self.features(x)\n\n\nclass residualBlock(nn.Module):\n    def __init__(self, in_channels=64, k=3, n=64, s=1):\n        super(residualBlock, self).__init__()\n\n        self.conv1 = nn.Conv2d(in_channels, n, k, stride=s, padding=1)\n        self.bn1 = nn.BatchNorm2d(n)\n        self.conv2 = nn.Conv2d(n, n, k, stride=s, padding=1)\n        self.bn2 = nn.BatchNorm2d(n)\n\n    def forward(self, x):\n        y = sigmoid_mul(self.bn1(self.conv1(x)))\n        return self.bn2(self.conv2(y)) + x\n\nclass upsampleBlock(nn.Module):\n    # Implements resize-convolution\n    def __init__(self, in_channels, out_channels):\n        super(upsampleBlock, self).__init__()\n       

### Loss Functions

In [0]:
import torchvision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#generator = Generator(16, 2)
generator = GeneratorRRDB(3, filters=64, num_res_blocks=2).to(device)
discriminator = Discriminator(input_shape=(3,256,256)).to(device)
#discriminator = Discriminator()
feature_extractor = FeatureExtractor().to(device)

# set feature extractor to inference mode
feature_extractor.eval()

criterion_GAN = torch.nn.BCEWithLogitsLoss().to(device)G = torch.load(model_dir + 'generator_param.pth')
criterion_content = torch.nn.L1Loss().to(device)
criterion_pixel = torch.nn.L1Loss().to(device)




Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:08<00:00, 71.6MB/s]


### Optimizer

In [0]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(lr_b1, lr_b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(lr_b1, lr_b2))

## Preprocessing

### Setting device to use for tensor operations

In [0]:
train_data = DatasetFromFolder(dataset, direction=direction)
train_data_loader = torch.utils.data.DataLoader(dataset=train_data,
                                                batch_size=1,
                                                shuffle=True)
test_data = DatasetFromFolder(dataset, subfolder="test", direction=direction)
test_data_loader = torch.utils.data.DataLoader(dataset=test_data,
                                                batch_size=1,
                                                shuffle=True)

### Initializing weights (if required)

In [0]:
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor


## Training

In [0]:
i_,j_ = 0, 0
from torch.autograd import Variable
for epoch in range(40):
    for i, (input_, target) in enumerate(train_data_loader):
        imgs_lr, imgs_hr = Variable(input_.type(Tensor)), Variable(target.type(Tensor))
        iterations = epoch*len(train_data_loader)+i 
        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((imgs_lr.size(0), *discriminator.output_shape))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((imgs_lr.size(0), *discriminator.output_shape))), requires_grad=False)
        optimizer_G.zero_grad()
        gen_hr = generator(imgs_lr)
        #print(gen_hr.size())
        loss_pixel = criterion_pixel(gen_hr, imgs_hr)
        if iterations < 500:
            i_ += 1

            loss_pixel.backward()
            optimizer_G.step()
            if i_ > 800:
                print(
                    "[Epoch %d/%d] [Batch %d/%d] [G pixel: %f]"
                    % (epoch, 40, i, len(train_data_loader), loss_pixel.item())
                )
                i_ = 0
            continue

        pred_real = discriminator(imgs_hr).detach()
        pred_fake = discriminator(gen_hr)

        loss_GAN = criterion_GAN(pred_fake-pred_real.mean(0, keepdim=True), valid)

        # content loss
        gen_features = feature_extractor(gen_hr)
        real_features = feature_extractor(imgs_hr).detach()
        loss_content = criterion_content(gen_features, real_features)

        loss_G = loss_content + 5e-3*loss_GAN + 1e-2*loss_pixel
        loss_G.backward()
        optimizer_G.step()

        optimizer_D.zero_grad()
        pred_real = discriminator(imgs_hr)
        pred_fake = discriminator(gen_hr.detach())

        loss_real = criterion_GAN(pred_real - pred_fake.mean(0, keepdim=True), valid)
        loss_fake = criterion_GAN(pred_fake - pred_real.mean(0, keepdim=True), fake)

        loss_D = (loss_real + loss_fake)/2
        loss_D.backward()
        optimizer_D.step()
        j_ += 1
        if j_ > 1000:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, content: %f, adv: %f, pixel: %f]"
                % (
                    epoch,
                    40,
                    i,
                    len(train_data_loader),
                    loss_D.item(),
                    loss_G.item(),
                    loss_content.item(),
                    loss_GAN.item(),
                    loss_pixel.item(),
                )
            )
            j_ = 0
    torch.save(generator, model_dir + 'generator_param.pth')
    torch.save(discriminator, model_dir + 'discriminator_param.pth')


[Epoch 0/40] [Batch 1500/8720] [D loss: 0.000055] [G loss: 2.451745, content: 2.361094, adv: 13.708691, pixel: 2.210711]
[Epoch 0/40] [Batch 2501/8720] [D loss: 0.000019] [G loss: 1.970237, content: 1.876372, adv: 13.642138, pixel: 2.565462]
[Epoch 0/40] [Batch 3502/8720] [D loss: 0.000014] [G loss: 2.366439, content: 2.254105, adv: 14.364664, pixel: 4.051015]
[Epoch 0/40] [Batch 4503/8720] [D loss: 0.000003] [G loss: 2.045160, content: 1.943473, adv: 15.943725, pixel: 2.196787]
[Epoch 0/40] [Batch 5504/8720] [D loss: 0.000012] [G loss: 2.369820, content: 2.229564, adv: 23.037378, pixel: 2.506909]
[Epoch 0/40] [Batch 6505/8720] [D loss: 0.000008] [G loss: 2.430858, content: 2.293442, adv: 23.135441, pixel: 2.173901]
[Epoch 0/40] [Batch 7506/8720] [D loss: 0.000003] [G loss: 2.817745, content: 2.680782, adv: 24.021860, pixel: 1.685330]
[Epoch 0/40] [Batch 8507/8720] [D loss: 0.000016] [G loss: 2.021290, content: 1.854536, adv: 26.510788, pixel: 3.419985]


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GeneratorRRDB. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResidualInResidualDenseBlock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "

[Epoch 1/40] [Batch 788/8720] [D loss: 0.000010] [G loss: 1.579724, content: 1.439387, adv: 22.835571, pixel: 2.615944]
[Epoch 1/40] [Batch 1789/8720] [D loss: 0.000002] [G loss: 1.421168, content: 1.282766, adv: 24.660931, pixel: 1.509767]
[Epoch 1/40] [Batch 2790/8720] [D loss: 0.000004] [G loss: 1.360244, content: 1.245099, adv: 20.789249, pixel: 1.119895]
[Epoch 1/40] [Batch 3791/8720] [D loss: 0.000003] [G loss: 0.830386, content: 0.695942, adv: 25.081345, pixel: 0.903750]
[Epoch 1/40] [Batch 4792/8720] [D loss: 0.001739] [G loss: 0.650007, content: 0.597509, adv: 10.049887, pixel: 0.224910]
[Epoch 1/40] [Batch 5793/8720] [D loss: 0.002791] [G loss: 0.868588, content: 0.830222, adv: 7.285985, pixel: 0.193555]
[Epoch 1/40] [Batch 6794/8720] [D loss: 0.001043] [G loss: 0.505066, content: 0.453003, adv: 10.055607, pixel: 0.178439]
[Epoch 1/40] [Batch 7795/8720] [D loss: 0.000094] [G loss: 457956.375000, content: 450030.937500, adv: 11.806261, pixel: 792535.937500]
[Epoch 2/40] [Batch

## Evaluation

In [0]:
def fix(image: np.ndarray) -> np.ndarray:
    """
    This function should take a degraded image in BGR format as a 250x250x3
    numpy array with dtype np.uint8, and return its fixed version in the same format.

    Incorrect formats will make the image look completely wrong.
    """

# Results
Run this block after done to look at some of the results of the fix function yourself.

In [0]:
%matplotlib inline

import os
import random
from glob import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np

NUM_DISPLAY = 5

files = glob('/content/rephrase-pubfig831/correct/test/*/*')
grid = []

for path in random.sample(files, NUM_DISPLAY):
  correct = cv2.imread(path)
  split = path.split('/')
  degraded = cv2.imread('/'.join([*split[:3], 'degraded', *split[4:]]))
  fixed = fix(degraded)
  grid.append(np.column_stack([degraded, fixed, correct]))

image = np.row_stack(grid)
dpi = float(plt.rcParams['figure.dpi'])
figsize = image.shape[1] / dpi, image.shape[0] / dpi
ax = plt.figure(figsize=figsize).add_axes([0, 0, 1, 1])
ax.axis('off')
ax.imshow(image[..., ::-1])
plt.show()